In [2]:
from TCFile import TCFile
import numpy as np
import napari
import h5py
from skimage import morphology, filters, feature, measure, restoration, segmentation
from tqdm import tqdm
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import copy
from scipy import ndimage

def circular_filter(array_shape, pixel_radius):
    filter = np.zeros(array_shape)
    N = array_shape[0]
    x, y = np.arange(N), np.arange(N)
    X, Y = np.meshgrid(x, y)
    filter[(X-N//2)**2 + (Y-N//2)**2 < pixel_radius**2] = 1
    return filter

def fourier_filter_with_thresholding(image, threshold, filter):

    temp = np.zeros(image.shape)
    temp = temp + image
    temp[image<threshold] = 0
    ###############################
    temp = filters.gaussian(temp)

    a_fourier = np.fft.fftshift(np.fft.fft2(temp))
    H, W = a_fourier.shape
    a_fourier[H//2:H//2 + 1, W//2:W//2 + 1]=0
    a_fourier = a_fourier * filter
    a_mod = np.fft.ifft2(a_fourier)
    ################################
    return np.abs(a_mod)

def get_mip_with_noise_reduction(data, first_thr_idx=0, second_thr_idx=1, fourier_filter_radius=400):
    # Get MIP image
    img_max = np.max(data, axis=0)
    #######################################################
    # Noise reduction
    thr = filters.threshold_multiotsu(img_max, 4)
    aa = fourier_filter_with_thresholding(img_max, thr[first_thr_idx], fourier_filter_radius)
    # Thresholding for better clarity
    aa_thr = filters.threshold_multiotsu(aa)
    aa[aa<aa_thr[second_thr_idx]] = 0
    # Mask generating and applying remove small objects
    mask = aa.copy()
    mask = morphology.remove_small_objects(mask.astype(bool), 1000)
    aa[mask==0] = 0
    # mask initialization for memory
    mask = 0
    #######################################################
    return aa


def separate_distance_map_with_centroids(distance_map, centroid1, centroid2):
    
    points_idx = np.array(np.where(distance_map!=0))
    
    # Calculate vector, midpoint, and perpendicular vector
    vector = centroid2 - centroid1
    midpoint = (centroid1 + centroid2) / 2

    def classify_point(point, midpoint, vector):
        relative_position = point - midpoint
        dot_product = np.dot(relative_position, vector)
        return 'left' if dot_product > 0 else 'right'

    classified_points = {'left': [], 'right': []}

    for p_idx in range(points_idx.shape[1]):
        point = np.array([points_idx[0][p_idx], points_idx[1][p_idx]])
        side = classify_point(point, midpoint, vector)
        classified_points[side].append(point)

    # Convert classified points to numpy arrays for plotting
    classified_points['left'] = np.array(classified_points['left'])
    classified_points['right'] = np.array(classified_points['right'])
    
    separated_distance_map = np.zeros(distance_image.shape)
    for l in classified_points['left']:
        separated_distance_map[l[0], l[1]] = 1

    for r in classified_points['right']:
        separated_distance_map[r[0], r[1]] = 2
        
    return separated_distance_map

In [3]:
viewer = napari.Viewer()

In [6]:
dir = r"C:\rkka_Projects\cell_death_v1\Data\pathway\raw\Apoptosis"
file = r"/compressed_230512.160429.CD95_TNF_Ctr.003.CD95.A1.T001P12.TCF"

tcfile = TCFile(dir + file, '3D')

In [7]:
data = np.zeros((36, 1009, 1009))
data_stack = np.zeros((36, 1009, 1009))

for i in tqdm(range(36)):
    # 1. 로딩
    img = tcfile[i]
    img_max = np.max(img, axis=0)
    data[i] = img_max
    
    aa = get_mip_with_noise_reduction(img, first_thr_idx=0, second_thr_idx=1)
    img_max[aa==0]=0
    data_stack[i] = img_max

100%|██████████| 36/36 [00:35<00:00,  1.00it/s]


In [8]:
viewer.add_image(data)
viewer.add_image(data_stack)

<Image layer 'data_stack' at 0x1fc89bcd010>

In [192]:
temp = data_stack.copy()
for i in range(36):
    
    temp[i] = ndimage.distance_transform_edt(temp[i])
    thr = filters.threshold_multiotsu(temp[i], 4)
    temp[i][temp[i]<thr[1]] = 0
    

In [141]:
viewer.add_image(temp)

<Image layer 'temp' at 0x236640d5eb0>

In [193]:
temp_labels = measure.label(temp.astype(bool))
props = measure.regionprops_table(temp_labels, properties=['label', 'bbox', 'centroid'])
props = pd.DataFrame(props)
props = props.drop(index=props[props['bbox-0']!=0].index)
props = props.drop(index=props[props['bbox-3']!=36].index)
props = props.drop(columns=['centroid-0'])
props

,label,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,centroid-1,centroid-2
0,1,0,0,522,36,133,696,34.983106,617.281638
1,2,0,56,59,36,218,351,129.011914,215.673233
2,3,0,149,452,36,430,625,264.786273,559.691364
3,4,0,157,722,36,335,866,244.509297,793.566544
4,5,0,148,0,36,281,197,215.082517,81.818904
6,7,0,407,741,36,549,977,482.499478,843.898815
7,8,0,422,368,36,780,591,609.527608,473.588143
8,9,0,565,164,36,911,405,764.189071,288.186827
9,10,0,564,16,36,787,169,689.382205,81.477456
10,11,0,630,705,36,865,868,758.161699,773.172198


In [194]:
z = np.zeros((36, 1009, 1009))
for i in range(props.shape[0]):
    z = z + (temp_labels==props.iloc[i]['label']).astype(int)
    
labels = measure.label(z)

In [195]:
viewer.add_labels(labels)

<Labels layer 'labels' at 0x236646b8a90>

In [200]:
data[0, 750, 259]

1.3428000211715698

In [196]:
dtemp = data_stack.copy()
for i in range(36):
    for k in range(10):
        dtemp[i] = morphology.erosion(dtemp[i])

In [197]:
from skimage import segmentation
la = np.zeros((36, 1009, 1009))
for i in range(36):
    la[i] = segmentation.watershed(data_stack[i], markers=labels[i], mask=dtemp.astype(bool)[i])

In [198]:
viewer.add_labels(la.astype(int))

<Labels layer 'Labels' at 0x235a9a15d00>